In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

How many renter households make between 2,000 and 4,000 monthly income in 2023?

In [11]:
# load data: ownership, household income, monthly gross rent 2023 in chicago
# https://usa.ipums.org/usa-action/variables/HHINCOME#description_section
# sample is the 2023 ACS sample from IPUMS: https://usa.ipums.org/usa/sampdesc.shtml#us2023a
df = pd.read_csv('source/ipums 012724 extract.csv')

In [12]:
df.head()

,YEAR,SAMPLE,SERIAL,CBSERIAL,HHWT,CLUSTER,CPI99,CITY,STRATA,GQ,OWNERSHP,OWNERSHPD,RENTGRS,HHINCOME
0,2023,202301,452359,2023010000392,32.0,2023004523591,0.547,1190,316817,3,0,0,0,9999999
1,2023,202301,452363,2023010000588,82.0,2023004523631,0.547,1190,316017,4,0,0,0,9999999
2,2023,202301,452364,2023010000609,68.0,2023004523641,0.547,1190,316317,3,0,0,0,9999999
3,2023,202301,452365,2023010000622,29.0,2023004523651,0.547,1190,316117,3,0,0,0,9999999
4,2023,202301,452371,2023010000872,13.0,2023004523711,0.547,1190,315117,4,0,0,0,9999999


In [13]:
# remove duplicate household serials
df.drop_duplicates(subset=['SERIAL'], keep='first', inplace=True)

In [14]:
# calculate monthly hh income
# filter out 9999999 = N/A
df['monthly_hh_income'] = np.where(df['HHINCOME']!=9999999,df['HHINCOME']/12, np.nan)
df['monthly_hh_income'].describe()

count      7837.000000
mean       9992.769002
std       11575.739544
min        -110.833333
25%        3166.666667
50%        6916.666667
75%       12500.000000
max      145333.333333
Name: monthly_hh_income, dtype: float64

In [15]:
# bin monthly income
bins = [0,1,2000,4000,6000,8000,10000,12000,100000000]
labels = [f'${bins[i]}-{bins[i+1]}' for i in range(len(bins)-1)]
df['income_bin'] = pd.cut(df['monthly_hh_income'], bins=bins, labels=labels, include_lowest=True)

About 129,000 renter households in Chicago make between 2,000 and 4,000 a month, according to a WBEZ analysis of Census data. 

In [16]:
# ownership code = 2 means renters https://usa.ipums.org/usa-action/variables/OWNERSHP#codes_section
# number of renter households by monthly income bucket
g = df[df['OWNERSHP'] == 2].groupby('income_bin')['HHWT'].sum().reset_index()
g

/tmp/ipykernel_5261/1993253848.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  g = df[df['OWNERSHP'] == 2].groupby('income_bin')['HHWT'].sum().reset_index()


,income_bin,HHWT
0,$0-1,22286.0
1,$1-2000,139888.0
2,$2000-4000,128610.0
3,$4000-6000,99300.0
4,$6000-8000,76662.0
5,$8000-10000,52589.0
6,$10000-12000,34228.0
7,$12000-100000000,80404.0


In [17]:
# check total number of renter households is roughly 600k 
# https://www.housingstudies.org/releases/2023-state-rental-housing-city-chicago/#:~:text=Previous%20IHS%20reports%20have%20highlighted,challenges%2C%20particularly%20in%20certain%20submarkets.

g['HHWT'].sum()

633967.0

How much of their income goes towards rent?

In [18]:
# calculate rent burdens
df['rent_burden'] = df['RENTGRS']/df['monthly_hh_income']
df['rent_burden_half'] = np.where(df['rent_burden'] >= 0.5, 'half or more', 'less than half')

About one third of them spend more than half their income on rent and utilities. 

In [19]:
# number of very rent burdened renter households by income bin
p = pd.pivot_table(df[df['OWNERSHP'] == 2],
              index='income_bin',
              columns='rent_burden_half',
              values='HHWT',
              aggfunc='sum').reset_index()

p['total'] = p['less than half'] + p['half or more']
p['pct'] = p['half or more']/p['total']
p

/tmp/ipykernel_5261/584029573.py:2: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  p = pd.pivot_table(df[df['OWNERSHP'] == 2],


rent_burden_half,income_bin,half or more,less than half,total,pct
0,$0-1,18761.0,3525.0,22286.0,0.841829
1,$1-2000,100108.0,39780.0,139888.0,0.715630
2,$2000-4000,38894.0,89716.0,128610.0,0.302418
3,$4000-6000,3490.0,95810.0,99300.0,0.035146
4,$6000-8000,1057.0,75605.0,76662.0,0.013788
5,$8000-10000,643.0,51946.0,52589.0,0.012227
6,$10000-12000,0.0,34228.0,34228.0,0.000000
7,$12000-100000000,0.0,80404.0,80404.0,0.000000


IPUMS v3 includes household income, built year 1 and 2, bedrooms, rent, gross rent for 1980, 1990, 2000, 2005 (ACS), 2010 (ACS), 2014 (ACS), 2023 (ACS) 

In [72]:
# load ipums data
df = pd.read_csv('source/ipums_v3.csv', low_memory=False)
df.head(3)

,YEAR,SAMPLE,SERIAL,CBSERIAL,HHWT,CLUSTER,CPI99,CITY,STRATA,GQ,OWNERSHP,OWNERSHPD,RENT,RENTGRS,HHINCOME,BUILTYR,BUILTYR2,BEDROOMS
0,1980,198002,550665,NaN,100,1980005506652,2.295,1190,21,1,2,22,165,165,16005,7.0,NaN,2
1,1980,198002,550666,NaN,100,1980005506662,2.295,1190,21,1,2,22,212,235,18005,5.0,NaN,2
2,1980,198002,550667,NaN,100,1980005506672,2.295,1190,35,1,2,22,325,415,12535,6.0,NaN,5


In [73]:
df['YEAR'] = df['YEAR'].astype(str)

In [74]:
# adjust rents and incomes for inflation with these constants: https://usa.ipums.org/usa/cpi99.shtml
df['adj_rent'] = (df['RENTGRS'] * df['CPI99']) / 0.547 # adjust to 1999 then 2023
df['adj_hhincome'] = (df['HHINCOME'] * df['CPI99']) / 0.547

In [75]:
# remove duplicate household serials
df.drop_duplicates(subset=['SERIAL'], keep='first', inplace=True)

In [76]:
# check total households looks right?
df.groupby('YEAR')['HHWT'].sum()

YEAR
1980    1140300
1990    1007025
2000    1124221
2005     868164
2010    1055521
2014     880277
2023    1170677
Name: HHWT, dtype: int64

In [77]:
# pivot by adjusted rent
pivot = pd.pivot_table(df[df['adj_rent'] != 0],
              index='adj_rent',
              columns='YEAR',
              values='HHWT',
              aggfunc='sum')

pivot.to_csv('processed/rent_pivot_by_year.csv')

pivot = pivot.reset_index()
pivot

YEAR,adj_rent,1980,1990,2000,2005,2010,2014,2023
0,5.148080,NaN,NaN,NaN,NaN,NaN,209.0,NaN
1,18.281536,NaN,NaN,108.0,NaN,NaN,NaN,NaN
2,20.109689,NaN,NaN,123.0,NaN,NaN,NaN,NaN
3,25.594150,NaN,NaN,124.0,NaN,NaN,NaN,NaN
4,27.934186,NaN,NaN,NaN,NaN,150.0,NaN,NaN
...,...,...,...,...,...,...,...,...
4081,5468.000000,NaN,NaN,NaN,NaN,NaN,NaN,99.0
4082,5700.000000,NaN,NaN,NaN,NaN,NaN,NaN,83.0
4083,5780.000000,NaN,NaN,NaN,NaN,NaN,NaN,79.0
4084,5800.000000,NaN,NaN,NaN,NaN,NaN,NaN,95.0


In [78]:
# load 2023-adjusted AMIs for each year
# source: NHGIS
# https://docs.google.com/spreadsheets/d/1C3ToVnNv3JRd01gKtjIfcY_IeDyKyOS1pvqflSLynAU/edit?usp=sharing

ami = {'1980': 64197, # decennial
       '1990': 64623, # decennial
       '2000': 70612, # decennial
       '2010': 62539, # acs 1-year,
       '2014': 62622, #acs 1-year
       '2023': 74474} # acs 1-year

In [79]:
# create a df that has the % of affordable units at each 10% AMI interval for all years

ami_pcts = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2]

dfdict = {'year': [],
         'ami': [],
         'ami_pct': [],
         'ami_pct_value': [],
         'ami_pct_monthly': [],
         'ami_pct_aff_threshold': [],
         'aff_units': [],
         'pct_aff_units': []
         }

for year in ami.keys():
    for pct in ami_pcts:
        dfdict['year'].append(year)
        dfdict['ami'].append(ami[year])
        dfdict['ami_pct'].append(pct)
        
        ami_pct_value = ami[year] * pct # %ami
        dfdict['ami_pct_value'].append(ami_pct_value)
        
        ami_pct_monthly = ami_pct_value / 12 # %ami per month
        dfdict['ami_pct_monthly'].append(ami_pct_monthly)
        
        ami_pct_aff_threshold = ami_pct_monthly * 0.3 # 30% of monthly income
        dfdict['ami_pct_aff_threshold'].append(ami_pct_aff_threshold)

        aff_units = pivot.loc[pivot['adj_rent'] < ami_pct_aff_threshold, year].sum() # calc num of rentals less than threshold
        dfdict['aff_units'].append(aff_units)

        pct_aff_units = aff_units / pivot[year].sum()
        dfdict['pct_aff_units'].append(pct_aff_units)

In [80]:
output = pd.DataFrame(dfdict)
output.to_csv('output/aff_units_by_ami_threshold.csv')
output

,year,ami,ami_pct,ami_pct_value,ami_pct_monthly,ami_pct_aff_threshold,aff_units,pct_aff_units
0,1980,64197,0.1,6419.7,534.975000,160.4925,3100.0,0.004885
1,1980,64197,0.2,12839.4,1069.950000,320.9850,29000.0,0.045698
2,1980,64197,0.3,19259.1,1604.925000,481.4775,50500.0,0.079578
3,1980,64197,0.4,25678.8,2139.900000,641.9700,93300.0,0.147022
4,1980,64197,0.5,32098.5,2674.875000,802.4625,186900.0,0.294516
...,...,...,...,...,...,...,...,...
115,2023,74474,1.6,119158.4,9929.866667,2978.9600,560210.0,0.941672
116,2023,74474,1.7,126605.8,10550.483333,3165.1450,566074.0,0.951529
117,2023,74474,1.8,134053.2,11171.100000,3351.3300,572573.0,0.962453
118,2023,74474,1.9,141500.6,11791.716667,3537.5150,577674.0,0.971028


In [95]:
# create a df that has a % of affordable units at each ami group

dfdict = {'year': [],
         'ami': [],
         'ami_pct_label': [],
         'aff_units': [],
         'pct_aff_units': []
         }

for year in ami.keys():
    for pct in [0, 0.5, 1, 1.5, 2]: # intervals of 50% up to 250% 
        dfdict['year'].append(year)
        dfdict['ami'].append(ami[year])
        
        ami_pct_value_lower = ami[year] * pct # %ami
        ami_pct_value_upper = ami[year] * (pct + 0.5) 
        ami_pct_label = f'{pct}-{pct+0.5}%'
        dfdict['ami_pct_label'].append(ami_pct_label)
        
        ami_pct_aff_threshold_lower = (ami_pct_value_lower / 12) * 0.3 # %30% of ami montly income is the aff threshold
        ami_pct_aff_threshold_upper = (ami_pct_value_upper / 12) * 0.3 

        aff_units = pivot.loc[(pivot['adj_rent'] < ami_pct_aff_threshold_upper) & (pivot['adj_rent'] >= ami_pct_aff_threshold_lower), year].sum() # inclusive of lower bound, exclusive of upper bound
        dfdict['aff_units'].append(aff_units)

        pct_aff_units = aff_units / pivot[year].sum()
        dfdict['pct_aff_units'].append(pct_aff_units)

In [96]:
output = pd.DataFrame(dfdict)
output

,year,ami,ami_pct_label,aff_units,pct_aff_units
0,1980,64197,0-0.5%,186900.0,0.294516
1,1980,64197,0.5-1.0%,397200.0,0.625906
2,1980,64197,1-1.5%,45100.0,0.071068
3,1980,64197,1.5-2.0%,5200.0,0.008194
4,1980,64197,2-2.5%,200.0,0.000315
5,1990,64623,0-0.5%,133935.0,0.231004
6,1990,64623,0.5-1.0%,358770.0,0.618788
7,1990,64623,1-1.5%,69780.0,0.120353
8,1990,64623,1.5-2.0%,11700.0,0.020180
9,1990,64623,2-2.5%,5610.0,0.009676


## scratch

In [43]:
# pivot by adjusted rent

bed_2_3 = df.loc[
(df['adj_rent'] != 0) & 
df['BEDROOMS'].isin([
    2,
    3])]

pivot_bed_2_3 = pd.pivot_table(bed_2_3,
              index='adj_rent',
              columns='YEAR',
              values='HHWT',
              aggfunc='sum')

#pivot.to_csv('processed/rent_pivot_by_year.csv')

pivot_bed_2_3 = pivot_bed_2_3.reset_index()
pivot_bed_2_3

YEAR,adj_rent,1980,1990,2000,2005,2010,2014,2023
0,18.281536,NaN,NaN,108.0,NaN,NaN,NaN,NaN
1,20.109689,NaN,NaN,123.0,NaN,NaN,NaN,NaN
2,37.425960,NaN,NaN,NaN,295.0,NaN,NaN,NaN
3,40.000000,NaN,NaN,NaN,NaN,NaN,NaN,40.0
4,41.901280,NaN,NaN,NaN,NaN,117.0,NaN,NaN
...,...,...,...,...,...,...,...,...
3385,5350.000000,NaN,NaN,NaN,NaN,NaN,NaN,143.0
3386,5468.000000,NaN,NaN,NaN,NaN,NaN,NaN,99.0
3387,5700.000000,NaN,NaN,NaN,NaN,NaN,NaN,83.0
3388,5800.000000,NaN,NaN,NaN,NaN,NaN,NaN,95.0


In [45]:
# Define bins and labels
bins = [0, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000, 1100, 1200, 1300, 1400, 
        1500, 1600, 1700, 1800, 1900, 2000, 2100, 2200, 2300, 2400, 2500, 2600, 2700,
        2800, 2900, 3000, 3100, 3200, 3300, 3400, 3500, 3600, 3700, 3800, 3900, 4000, 7000]  # Edges of bins

labels = [f'${bins[i]}-{bins[i+1]}' for i in range(len(bins)-1)]
print(labels)

# Apply binning
pivot['adj_rent_bin'] = pd.cut(pivot['adj_rent'], bins=bins, labels=labels, include_lowest=True)
pivot_bed_2_3['adj_rent_bin'] = pd.cut(pivot_bed_2_3['adj_rent'], bins=bins, labels=labels, include_lowest=True)

['$0-100', '$100-200', '$200-300', '$300-400', '$400-500', '$500-600', '$600-700', '$700-800', '$800-900', '$900-1000', '$1000-1100', '$1100-1200', '$1200-1300', '$1300-1400', '$1400-1500', '$1500-1600', '$1600-1700', '$1700-1800', '$1800-1900', '$1900-2000', '$2000-2100', '$2100-2200', '$2200-2300', '$2300-2400', '$2400-2500', '$2500-2600', '$2600-2700', '$2700-2800', '$2800-2900', '$2900-3000', '$3000-3100', '$3100-3200', '$3200-3300', '$3300-3400', '$3400-3500', '$3500-3600', '$3600-3700', '$3700-3800', '$3800-3900', '$3900-4000', '$4000-7000']


In [46]:
pivot_bed_2_3

YEAR,adj_rent,1980,1990,2000,2005,2010,2014,2023,adj_rent_bin
0,18.281536,NaN,NaN,108.0,NaN,NaN,NaN,NaN,$0-100
1,20.109689,NaN,NaN,123.0,NaN,NaN,NaN,NaN,$0-100
2,37.425960,NaN,NaN,NaN,295.0,NaN,NaN,NaN,$0-100
3,40.000000,NaN,NaN,NaN,NaN,NaN,NaN,40.0,$0-100
4,41.901280,NaN,NaN,NaN,NaN,117.0,NaN,NaN,$0-100
...,...,...,...,...,...,...,...,...,...
3385,5350.000000,NaN,NaN,NaN,NaN,NaN,NaN,143.0,$4000-7000
3386,5468.000000,NaN,NaN,NaN,NaN,NaN,NaN,99.0,$4000-7000
3387,5700.000000,NaN,NaN,NaN,NaN,NaN,NaN,83.0,$4000-7000
3388,5800.000000,NaN,NaN,NaN,NaN,NaN,NaN,95.0,$4000-7000


In [48]:
plot_data = pivot_bed_2_3.groupby('adj_rent_bin', observed=True)[['1980', '1990', '2000', '2005', '2010', '2014', '2023']].sum().reset_index()
plot_data

YEAR,adj_rent_bin,1980,1990,2000,2005,2010,2014,2023
0,$0-100,0.0,0.0,1879.0,1091.0,621.0,1010.0,472.0
1,$100-200,6700.0,6840.0,6027.0,1714.0,2457.0,2971.0,2349.0
2,$200-300,13000.0,12510.0,12678.0,8612.0,8770.0,9124.0,9828.0
3,$300-400,9400.0,11310.0,7761.0,5020.0,6411.0,6546.0,6827.0
4,$400-500,8100.0,11205.0,8228.0,7062.0,6060.0,5946.0,6400.0
5,$500-600,16300.0,10590.0,9484.0,6803.0,6793.0,7043.0,4553.0
6,$600-700,31200.0,12720.0,15377.0,6834.0,10583.0,7281.0,6384.0
7,$700-800,43500.0,24300.0,21382.0,14175.0,9437.0,8824.0,18561.0
8,$800-900,61300.0,34305.0,29399.0,13477.0,17232.0,14396.0,20936.0
9,$900-1000,69800.0,37605.0,39269.0,22265.0,26855.0,27239.0,29031.0


In [49]:
# pct of renters living in 1 vs 2-3 beds
pd.pivot_table(df[df['adj_rent'] != 0],
              index='YEAR',
              columns='BEDROOMS',
              values='HHWT',
              aggfunc='sum')

BEDROOMS,1,2,3,4,5,6,7,10
YEAR,,,,,,,,
1980,63400.0,229000.0,226300.0,99200.0,14400.0,2300.0,NaN,NaN
1990,58935.0,199455.0,203220.0,104145.0,10980.0,3060.0,NaN,NaN
2000,71056.0,200143.0,202552.0,97175.0,15636.0,3481.0,NaN,NaN
2005,34399.0,140407.0,149090.0,87445.0,13432.0,4200.0,NaN,NaN
2010,53263.0,155634.0,192464.0,103039.0,16598.0,4434.0,NaN,2755.0
2014,34729.0,140175.0,171800.0,88354.0,14424.0,3237.0,NaN,2365.0
2023,72537.0,187992.0,214190.0,98059.0,16732.0,3773.0,1627.0,NaN
